# Delta CRUD
* Delta comes pre-installed on managed platform like Databricks, otherwise follow the instructions to install it
### Pre-requisites
* https://docs.delta.io/latest/quick-start.html

### Quickstart

In [0]:
#dbutils is a databricks utility function 
dbutils.fs.rm('/tmp/ch3', True)

Out[3]: False

In [0]:
data = spark.range(0, 5)
data.write.format("delta").mode("overwrite").save("/tmp/ch3/delta-table")

df = spark.read.format("delta").load("/tmp/ch3/delta-table")
df.show()

+---+
 id|
+---+
 2|
 0|
 3|
 1|
 4|
+---+

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "/tmp/ch3/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge( newData.alias("newData"), "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show(5)

+---+
 id|
+---+
 0|
 1|
 2|
 3|
 4|
+---+
only showing top 5 rows

In [0]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/ch3/delta-table")
df.show()

+---+
 id|
+---+
 0|
 1|
 3|
 4|
 2|
+---+

In [0]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/ch3/checkpoint").start("/tmp/ch3/delta-table")

In [0]:
stream2 = spark.readStream.format("delta").load("/tmp/ch3/delta-table").writeStream.format("console").start()

## Create a Delta Table

In [0]:
display(dbutils.fs.ls('/databricks-datasets/learning-spark-v2'))

path,name,size,modificationTime
dbfs:/databricks-datasets/learning-spark-v2/.DS_Store,.DS_Store,6148,1602174521000
dbfs:/databricks-datasets/learning-spark-v2/README/,README/,0,1655562813466
dbfs:/databricks-datasets/learning-spark-v2/SPARK_README.md,SPARK_README.md,3359,1577997727000
dbfs:/databricks-datasets/learning-spark-v2/blogs.json,blogs.json,915,1587070414000
dbfs:/databricks-datasets/learning-spark-v2/cctvVideos/,cctvVideos/,0,1655562813466
dbfs:/databricks-datasets/learning-spark-v2/flights/,flights/,0,1655562813466
dbfs:/databricks-datasets/learning-spark-v2/iot-devices/,iot-devices/,0,1655562813466
dbfs:/databricks-datasets/learning-spark-v2/loans/,loans/,0,1655562813466
dbfs:/databricks-datasets/learning-spark-v2/mnm_dataset.csv,mnm_dataset.csv,1284872,1587070414000
dbfs:/databricks-datasets/learning-spark-v2/people/,people/,0,1655562813466


In [0]:
dataPath = '/databricks-datasets/learning-spark-v2/people/people-10m.delta'
# dbutils.fs.ls(dataPath)

Out[7]: [FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/_delta_log/', name='_delta_log/', size=0, modificationTime=1655562638969),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00000-373539c8-e620-43e4-82e0-5eba22bb3b77-c000.snappy.parquet', name='part-00000-373539c8-e620-43e4-82e0-5eba22bb3b77-c000.snappy.parquet', size=27825521, modificationTime=1602174521000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00000-5e15f637-db75-425a-bc10-7af4bf04d7f1-c000.snappy.parquet', name='part-00000-5e15f637-db75-425a-bc10-7af4bf04d7f1-c000.snappy.parquet', size=27780910, modificationTime=1602100197000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00001-7ce85220-3ce6-4f30-bde3-9a05fbcdc424-c000.snappy.parquet', name='part-00001-7ce85220-3ce6-4f30-bde3-9a05fbcdc424-c000.snappy.parquet', size=27734182, modificationTime=1602100197000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00001-943ebb93-8446-4a6c-99f7-1ca12ec2511b-c000.snappy.parquet', name='part-00001-943ebb93-8446-4a6c-99f7-1ca12ec2511b-c000.snappy.parquet', size=27781558, modificationTime=1602174521000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00002-078d8630-60a4-48a0-9d10-44fd8a173b87-c000.snappy.parquet', name='part-00002-078d8630-60a4-48a0-9d10-44fd8a173b87-c000.snappy.parquet', size=27734188, modificationTime=1602174521000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00002-89e6d24b-94c8-4e84-8dd5-12b14ca26615-c000.snappy.parquet', name='part-00002-89e6d24b-94c8-4e84-8dd5-12b14ca26615-c000.snappy.parquet', size=27781560, modificationTime=1602100197000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00003-315a65b7-33f8-4ade-9333-b7a27f7878fd-c000.snappy.parquet', name='part-00003-315a65b7-33f8-4ade-9333-b7a27f7878fd-c000.snappy.parquet', size=27735048, modificationTime=1602174521000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00003-cbfb2b8d-a130-42f8-88ce-e9317d796959-c000.snappy.parquet', name='part-00003-cbfb2b8d-a130-42f8-88ce-e9317d796959-c000.snappy.parquet', size=27734190, modificationTime=1602100197000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00004-4403dea5-fbb8-4b6e-8d29-755d646706b4-c000.snappy.parquet', name='part-00004-4403dea5-fbb8-4b6e-8d29-755d646706b4-c000.snappy.parquet', size=28172443, modificationTime=1602100197000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00004-854d48c2-cfbf-4e02-9932-edba227c525d-c000.snappy.parquet', name='part-00004-854d48c2-cfbf-4e02-9932-edba227c525d-c000.snappy.parquet', size=28172441, modificationTime=1602174521000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00005-3d084889-621d-4800-8c24-be6ce540ed8b-c000.snappy.parquet', name='part-00005-3d084889-621d-4800-8c24-be6ce540ed8b-c000.snappy.parquet', size=27780908, modificationTime=1602174597000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00005-83ad6e0a-7069-4fa2-92dd-dd2e0624800c-c000.snappy.parquet', name='part-00005-83ad6e0a-7069-4fa2-92dd-dd2e0624800c-c000.snappy.parquet', size=26481810, modificationTime=1602100281000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00006-aa52f259-f2af-494a-b4ae-5ef02b2f0961-c000.snappy.parquet', name='part-00006-aa52f259-f2af-494a-b4ae-5ef02b2f0961-c000.snappy.parquet', size=27825523, modificationTime=1602100281000),
 FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/people/people-10m.delta/part-00006-dfdaedf2-5cb9-4ad5-9b11-1a88e962b76d-c000.snappy.parquet', name='part-00006-dfdaedf2-5cb9-4ad5-9b11-1a88e962b76d-c000.snappy

In [0]:

spark.sql(""" 
    CREATE TABLE IF NOT EXISTS some_delta_table  
    USING delta  
    OPTIONS (path = '{}') 
  """.format(dataPath)) 

Out[2]: DataFrame[]

## Read from Delta

In [0]:
%sql
SELECT COUNT(*) FROM some_delta_table 

count(1)
10000000


In [0]:
%sql
SELECT COUNT(*) FROM delta.`/databricks-datasets/learning-spark-v2/people/people-10m.delta`

count(1)
10000000


## Write to Delta

In [0]:
df.write.format("delta").saveAsTable("some_other_delta_table") 

## Update Delta Table

## Delete from Delta Table

## Convert to Delta

### From CSV

In [0]:
%fs head '/databricks-datasets/learning-spark-v2/mnm_dataset.csv'

[Truncated to first 65536 bytes]
State,Color,Count
TX,Red,20
NV,Blue,66
CO,Blue,79
OR,Blue,71
WA,Yellow,93
WY,Blue,16
CA,Yellow,53
WA,Green,60
OR,Green,71
TX,Green,68
NV,Green,59
AZ,Brown,95
WA,Yellow,20
AZ,Blue,75
OR,Brown,72
NV,Red,98
WY,Orange,45
CO,Blue,52
TX,Brown,94
CO,Red,82
CO,Red,12
CO,Red,17
OR,Green,16
AZ,Green,46
NV,Red,43
NM,Yellow,15
WA,Red,12
OR,Green,13
CO,Blue,95
WY,Red,63
TX,Orange,63
WY,Yellow,48
OR,Green,95
WA,Red,75
CO,Orange,93
NV,Orange,10
WY,Green,15
WA,Green,99
CO,Blue,98
CA,Green,86
UT,Red,92
AZ,Brown,16
CA,Red,100
UT,Red,77
TX,Yellow,29
WA,Orange,73
WY,Yellow,56
WA,Yellow,19
WY,Yellow,18
AZ,Orange,49
WA,Red,83
AZ,Red,59
CA,Blue,13
CO,Brown,53
CA,Red,23
CA,Yellow,49
NV,Blue,50
CO,Red,81
UT,Green,62
NM,Green,38
NV,Yellow,77
TX,Blue,49
WA,Brown,20
OR,Green,59
OR,Red,66
CA,Blue,34
UT,Blue,97
AZ,Blue,59
OR,Blue,29
CO,Red,17
AZ,Orange,54
TX,Green,69
NM,Brown,65
WY,Orange,23
NM,Green,42
OR,Green,39
WA,Orange,84
UT,Yellow,72
NV,Yellow,98
TX,Yellow,20
UT,Brown,58
CA,Red,67
CO,Orange,42
WA,Red,70
OR,Blue,54
TX,Green,85
WY,Brown,85
UT,Green,97
CO,Blue,52
WA,Yellow,43
WA,Yellow,77
WA,Green,10
TX,Red,97
WA,Brown,21
WY,Red,22
WY,Yellow,42
WA,Orange,30
NV,Orange,48
WA,Blue,59
CO,Orange,22
CA,Orange,25
CA,Yellow,69
WY,Blue,59
UT,Brown,77
TX,Green,82
CA,Orange,91
TX,Brown,96
WY,Blue,53
WA,Blue,42
TX,Green,74
AZ,Brown,79
OR,Orange,85
CO,Orange,21
AZ,Yellow,48
WA,Blue,37
NV,Blue,49
CA,Brown,95
CO,Brown,53
TX,Red,16
CO,Brown,56
NV,Red,90
WY,Brown,87
UT,Yellow,56
NM,Yellow,21
CA,Yellow,41
CA,Orange,71
UT,Yellow,97
CA,Brown,58
WY,Brown,43
NM,Blue,15
AZ,Yellow,72
WA,Brown,38
CO,Green,44
NV,Brown,65
AZ,Yellow,94
NV,Red,94
CA,Brown,92
WA,Orange,94
AZ,Orange,53
WY,Green,18
NM,Orange,69
CA,Blue,39
NM,Green,32
WA,Green,68
AZ,Green,77
UT,Red,46
NM,Green,87
CO,Red,78
UT,Blue,48
NV,Brown,10
NM,Orange,77
UT,Green,60
WA,Yellow,49
OR,Orange,18
OR,Yellow,72
CO,Orange,41
AZ,Blue,82
UT,Brown,78
UT,Orange,24
UT,Green,97
WA,Brown,26
CO,Brown,98
CO,Blue,14
WY,Yellow,78
UT,Orange,34
WA,Blue,18
CO,Blue,89
TX,Blue,80
UT,Yellow,81
NM,Yellow,75
WY,Green,10
CO,Red,79
UT,Brown,98
WA,Blue,84
NM,Blue,46
AZ,Blue,16
WA,Brown,17
TX,Blue,68
NM,Green,67
AZ,Green,89
OR,Brown,48
WY,Green,91
WA,Brown,52
AZ,Green,29
WY,Brown,30
NM,Red,50
NV,Brown,49
CA,Blue,99
WY,Orange,81
UT,Green,41
OR,Brown,35
TX,Red,13
WY,Blue,80
OR,Green,76
CO,Yellow,74
UT,Green,33
TX,Brown,55
CO,Green,22
WY,Orange,99
TX,Orange,93
TX,Yellow,83
OR,Green,77
CO,Yellow,37
CO,Blue,44
CA,Brown,80
WY,Brown,36
WA,Brown,26
NV,Green,49
AZ,Red,93
TX,Yellow,19
CO,Yellow,63
TX,Yellow,81
TX,Red,100
CA,Red,45
UT,Brown,59
NM,Brown,66
TX,Red,95
OR,Yellow,28
AZ,Yellow,83
CO,Red,52
NV,Red,23
WA,Green,27
WY,Yellow,90
NM,Yellow,29
CA,Green,73
NV,Brown,34
CA,Orange,32
TX,Red,44
WA,Brown,76
CO,Blue,90
CO,Red,82
NV,Orange,58
NM,Red,40
TX,Blue,19
NV,Orange,84
NM,Green,61
UT,Brown,92
AZ,Orange,50
AZ,Yellow,49
CA,Blue,65
NM,Red,50
WA,Yellow,57
TX,Yellow,64
CO,Red,94
AZ,Yellow,80
TX,Blue,98
OR,Brown,48
NM,Blue,98
AZ,Yellow,80
OR,Orange,91
CA,Brown,64
OR,Green,39
NV,Yellow,76
NM,Red,53
UT,Yellow,35
AZ,Blue,24
WY,Red,87
WY,Green,57
NM,Red,47
CO,Brown,57
WY,Green,97
WA,Yellow,31
WY,Brown,81
CA,Red,27
WA,Yellow,58
NM,Orange,70
WY,Green,39
WA,Red,38
NM,Brown,99
CA,Brown,94
CO,Yellow,14
CO,Green,25
NV,Red,52
WY,Brown,48
NV,Orange,65
AZ,Yellow,41
WY,Green,68
UT,Green,23
WA,Green,58
NM,Orange,41
WY,Orange,56
WY,Red,70
CO,Green,58
UT,Orange,36
CA,Red,50
WA,Red,10
CA,Yellow,42
AZ,Green,99
NM,Brown,36
UT,Green,77
TX,Green,91
WA,Orange,91
OR,Brown,16
CA,Orange,87
TX,Yellow,70
WA,Green,29
NM,Blue,70
TX,Red,55
WY,Green,86
OR,Blue,96
OR,Blue,37
WY,Blue,33
NM,Green,83
UT,Orange,87
WY,Yellow,62
WY,Red,23
TX,Brown,59
CO,Green,40

In [0]:
csvDataPath = '/tmp/chapter3/dataset.csv'

deltaDataPath = '/tmp/chapter3/csvToDelta'
parquetDataPath = '/tmp/chapter3/csvToParquet'

In [0]:
columns = ["State","Color", "Count"]
data = [("TX","Red",20), ("NV","Blue",66), ("CO","Blue",79),("CA","Yellow",53), ("WY","Orange",45)]

color_df = spark.sparkContext.parallelize(data).toDF(columns)
color_df.printSchema()

(color_df.write.option("header",True)
        .option("delimiter",",")
        .mode("overwrite")
        .csv(csvDataPath))

root
-- State: string (nullable = true)
-- Color: string (nullable = true)
-- Count: long (nullable = true)

In [0]:

(spark.read  
  .option('header', True)
  .option('inferSchema', True)
  .csv(csvDataPath)
  .write 
  .mode("overwrite") 
  .format("delta") 
  .partitionBy("Color") 
  .save(deltaDataPath) ) 

In [0]:
spark.sql("SELECT count(*) FROM delta.`{}`".format(deltaDataPath)).show()

+--------+
count(1)|
+--------+
 5|
+--------+

In [0]:
%sql
SHOW PARTITIONS delta.`/tmp/chapter3/csvToDelta`

Color
Orange
Blue
Yellow
Red


### From Parquet
<p><b>Step 1:</b> Convert Parquet to Delta  <br>
CONVERT TO DELTA <parquet path or table> <br>
[PARTITIONED BY (<partition columns if any>)] <br>

  <p><b>Step 2: </b>Optimize layout for fast queries: in addition to above conversion <br>
OPTIMIZE <delta_table> <br>
WHERE date >= current_timestamp() - INTERVAL 1 day <br>
ZORDER BY (<z order column/s>) <br>

In [0]:
%fs ls '/tmp/chapter3/csvToParquet'

path,name,size,modificationTime
dbfs:/tmp/chapter3/csvToParquet/Color=Blue/,Color=Blue/,0,1655566595388
dbfs:/tmp/chapter3/csvToParquet/Color=Orange/,Color=Orange/,0,1655566595388
dbfs:/tmp/chapter3/csvToParquet/Color=Red/,Color=Red/,0,1655566595388
dbfs:/tmp/chapter3/csvToParquet/Color=Yellow/,Color=Yellow/,0,1655566595388
dbfs:/tmp/chapter3/csvToParquet/_SUCCESS,_SUCCESS,0,1655566567000
dbfs:/tmp/chapter3/csvToParquet/_committed_721177383267740285,_committed_721177383267740285,35,1655566345000


In [0]:
parquet_df = (spark.read  
  .option('header', True)
  .option('inferSchema', True)
  .csv(csvDataPath)
  .write 
  .mode("overwrite") 
  .format("parquet") 
  .partitionBy("Color") 
  .save(parquetDataPath) ) 

In [0]:
spark.sql(""" 
    CREATE TABLE IF NOT EXISTS some_parquet_table  
    USING parquet  
    OPTIONS (path = '{}') 
  """.format(parquetDataPath)) 

Out[43]: DataFrame[]

In [0]:
%sql
select * from some_parquet_table

State,Count,Color


In [0]:
%sql
CONVERT TO DELTA some_parquet_table PARTITIONED BY (Color)

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input ')' expecting {'APPLY', 'CALLED', 'CHANGES', 'CLONE', 'COLLECT', 'CONTAINS', 'CONVERT', 'COPY', 'COPY_OPTIONS', 'CREDENTIAL', 'CREDENTIALS', 'DEEP', 'DEFINER', 'DELTA', 'DETERMINISTIC', 'ENCRYPTION', 'EXPECT', 'FAIL', 'FILES', 'FORMAT_OPTIONS', 'HISTORY', 'INCREMENTAL', 'INPUT', 'INVOKER', 'LANGUAGE', 'LIVE', 'MATERIALIZED', 'MODIFIES', 'OPTIMIZE', 'PATTERN', 'READS', 'RESTORE', 'RETURN', 'RETURNS', 'SAMPLE', 'SCD TYPE 1', 'SCD TYPE 2', 'SECURITY', 'SEQUENCE', 'SHALLOW', 'SNAPSHOT', 'SPECIFIC', 'SQL', 'STORAGE', 'STREAMING', 'UPDATES', 'UP_TO_DATE', 'VIOLATION', 'ZORDER', 'ADD', 'AFTER', 'ALL', 'ALTER', 'ALWAYS', 'ANALYZE', 'AND', 'ANTI', 'ANY', 'ARCHIVE', 'ARRAY', 'AS', 'ASC', 'AT', 'AUTHORIZATION', 'BETWEEN', 'BOTH', 'BUCKET', 'BUCKETS', 'BY', 'CACHE', 'CASCADE', 'CASE', 'CAST', 'CATALOG', 'CATALOGS', 'CHANGE', 'CHECK', 'CLEAR', 'CLUSTER', 'CLUSTERED', 'CODE', 'CODEGEN', 'COLLATE', 'COLLECTION', 'COLUMN', 'COLUMNS', 'COMMENT', 'COMMIT', 'COMPACT', 'COMPACTIONS', 'COMPUTE', 'CONCATENATE', 'CONSTRAINT', 'COST', 'CREATE', 'CROSS', 'CUBE', 'CURRENT', 'CURRENT_DATE', 'CURRENT_TIME', 'CURRENT_TIMESTAMP', 'CURRENT_USER', 'DAY', 'DATA', 'DATABASE', 'DATABASES', 'DATEADD', 'DATEDIFF', 'DBPROPERTIES', 'DEFAULT', 'DEFINED', 'DELETE', 'DELIMITED', 'DESC', 'DESCRIBE', 'DFS', 'DIRECTORIES', 'DIRECTORY', 'DISTINCT', 'DISTRIBUTE', 'DIV', 'DROP', 'ELSE', 'END', 'ESCAPE', 'ESCAPED', 'EXCEPT', 'EXCHANGE', 'EXISTS', 'EXPLAIN', 'EXPORT', 'EXTENDED', 'EXTERNAL', 'EXTRACT', 'FALSE', 'FETCH', 'FIELDS', 'FILTER', 'FILEFORMAT', 'FIRST', 'FN', 'FOLLOWING', 'FOR', 'FOREIGN', 'FORMAT', 'FORMATTED', 'FROM', 'FULL', 'FUNCTION', 'FUNCTIONS', 'GENERATED', 'GLOBAL', 'GRANT', 'GRANTS', 'GROUP', 'GROUPING', 'HAVING', 'HOUR', 'IDENTITY', 'IF', 'IGNORE', 'IMPORT', 'IN', 'INCREMENT', 'INDEX', 'INDEXES', 'INNER', 'INPATH', 'INPUTFORMAT', 'INSERT', 'INTERSECT', 'INTERVAL', 'INTO', 'IS', 'ITEMS', 'JOIN', 'KEY', 'KEYS', 'LAST', 'LATERAL', 'LAZY', 'LEADING', 'LEFT', 'LIKE', 'ILIKE', 'LIMIT', 'LINES', 'LIST', 'LOAD', 'LOCAL', 'LOCATION', 'LOCK', 'LOCKS', 'LOGICAL', 'MACRO', 'MAP', 'MATCHED', 'MERGE', 'MINUTE', 'MONTH', 'MSCK', 'NAMESPACE', 'NAMESPACES', 'NATURAL', 'NO', NOT, 'NULL', 'NULLS', 'OF', 'ON', 'ONLY', 'OPTION', 'OPTIONS', 'OR', 'ORDER', 'OUT', 'OUTER', 'OUTPUTFORMAT', 'OVER', 'OVERLAPS', 'OVERLAY', 'OVERWRITE', 'PARTITION', 'PARTITIONED', 'PARTITIONS', 'PERCENTILE_CONT', 'PERCENT', 'PIVOT', 'PLACING', 'POSITION', 'PRECEDING', 'PRIMARY', 'PRINCIPALS', 'PROPERTIES', 'PROVIDER', 'PROVIDERS', 'PURGE', 'QUALIFY', 'QUERY', 'RANGE', 'RECIPIENT', 'RECIPIENTS', 'RECORDREADER', 'RECORDWRITER', 'RECOVER', 'REDUCE', 'REFERENCES', 'REFRESH', 'REMOVE', 'RENAME', 'REPAIR', 'REPEATABLE', 'REPLACE', 'REPLICAS', 'RESET', 'RESPECT', 'RESTRICT', 'REVOKE', 'RIGHT', RLIKE, 'ROLE', 'ROLES', 'ROLLBACK', 'ROLLUP', 'ROW', 'ROWS', 'SECOND', 'SCHEMA', 'SCHEMAS', 'SELECT', 'SEMI', 'SEPARATED', 'SERDE', 'SERDEPROPERTIES', 'SESSION_USER', 'SET', 'MINUS', 'SETS', 'SHARE', 'SHARES', 'SHOW', 'SKEWED', 'SOME', 'SORT', 'SORTED', 'START', 'STATISTICS', 'STORED', 'STRATIFY', 'STRUCT', 'SUBSTR', 'SUBSTRING', 'SYNC', 'SYSTEM_TIME', 'SYSTEM_VERSION', 'TABLE', 'TABLES', 'TABLESAMPLE', 'TBLPROPERTIES', TEMPORARY, 'TERMINATED', 'THEN', 'TIME', 'TIMESTAMP', 'TIMESTAMPADD', 'TIMESTAMPDIFF', 'TO', 'TOUCH', 'TRAILING', 'TRANSACTION', 'TRANSACTIONS', 'TRANSFORM', 'TRIM', 'TRUE', 'TRUNCATE', 'TRY_CAST', 'TYPE', 'UNARCHIVE', 'UNBOUNDED', 'UNCACHE', 'UNION', 'UNIQUE', 'UNKNOWN', 'UNLOCK', 'UNSET', 'UPDATE', 'USE', 'USER', 'USING', 'VALUES', 'VERSION', 'VIEW', 'VIEWS', 'WHEN', 'WHERE', 'WINDOW', 'WITH', 'WITHIN', 'YEAR', 'ZONE', IDENTIFIER, BACKQUOTED_IDENTIFIER}(line 1, pos 57)

== SQL ==
CONVERT TO DELTA some_parquet_table PARTITIONED BY (Color)
---------------------------------------------------------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.w

In [0]:
%sql
OPTIMIZE color_delta_table 
WHERE date >= current_timestamp() - INTERVAL 1 day 
ZORDER BY (State) 